# Roteamento

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model='gpt-4o-mini')

prompt = ChatPromptTemplate.from_template('''Você é um professor de matemática de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_matematica = prompt | model

prompt = ChatPromptTemplate.from_template('''Você é um professor de física de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_fisica = prompt | model

prompt = ChatPromptTemplate.from_template('''Você é um professor de história de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_historia = prompt | model

prompt = ChatPromptTemplate.from_template('''{pergunta}''')
chain_generica = prompt | model

In [2]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template('Você deve categorizar a seguinte pergunta: {pergunta}')

class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos do ensino fundamental"""
    area_conhecimento: str = Field(description='A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser "física", "matemática" ou "história". Caso não se encaixe em nenhuma delas, retorne "outra"')

model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

Categorizador(area_conhecimento='história')

### Criando estrutura de roteamento

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

{'pergunta': 'Quando foi a inependencia dos estados unidos?',
 'categoria': Categorizador(area_conhecimento='história')}

In [4]:
def route(input):
    if input['categoria'].area_conhecimento == 'matemática':
        return chain_matematica
    if input['categoria'].area_conhecimento == 'física':
        return chain_fisica
    if input['categoria'].area_conhecimento == 'história':
        return chain_historia
    return chain_generica

In [5]:
chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
chain.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

AIMessage(content='A Independência dos Estados Unidos foi proclamada no dia 4 de julho de 1776. Para entender esse importante evento histórico, vamos dar uma olhada no contexto e nas razões que levaram à independência, assim como suas consequências.\n\n### Contexto Histórico\n\nDurante o século XVIII, as Treze Colônias britânicas na América do Norte estavam sob o domínio do Reino Unido. Essas colônias eram, na maioria, bastante prósperas e tinham desenvolvido uma certa autonomia ao longo dos anos. No entanto, a relação entre as colônias e a Grã-Bretanha começou a se deteriorar por várias razões:\n\n1. **Impostos Elevados**: Após a Guerra dos Sete Anos (1756-1763), a Grã-Bretanha estava endividada e decidiu aumentar os impostos nas colônias para arrecadar dinheiro. A famosa frase "sem impostos sem representação" reflete a insatisfação dos colonos, que não tinham representantes no Parlamento britânico.\n\n2. **Leis e Restrições**: O governo britânico impôs várias leis e regulamentações q

In [6]:
chain.invoke({'pergunta': 'Quanto é 1 + 21?'})

AIMessage(content='Claro! Vamos resolver a soma \\(1 + 21\\) de forma bem detalhada.\n\n### Passo 1: Compreender os números\n- O número \\(1\\) é um número inteiro e representa uma unidade.\n- O número \\(21\\) é também um número inteiro, e representa vinte e uma unidades.\n\n### Passo 2: Visualizar a soma\nQuando somamos, estamos basicamente agrupando quantidades. Então, quando somamos \\(1 + 21\\), estamos juntando uma unidade a um grupo que já tem vinte e uma unidades.\n\n### Passo 3: Realizar a soma\nAgora, vamos realizar a soma:\n- Começamos com \\(21\\).\n- Adicionamos \\(1\\) a \\(21\\).\n\n**Cálculo:**\n\\[\n21 + 1 = 22\n\\]\n\n### Passo 4: Resultado final\nPortanto, a soma de \\(1 + 21\\) é igual a \\(22\\).\n\n### Conclusão\nAssim, podemos concluir que:\n\\[\n1 + 21 = 22\n\\]\n\nSe você tiver mais alguma dúvida ou outra pergunta sobre matemática, fique à vontade para perguntar!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 247,